# Imbalanced Data: Fraud Detection

## 🎯 Цели

1. Понимание проблемы imbalanced data
2. Sampling techniques: SMOTE, undersampling
3. Cost-sensitive learning
4. Evaluation metrics
5. Практика: Credit Card Fraud (1:600 imbalance)

## 💼 Задача: Fraud Detection

**Extreme imbalance:** ~0.17% fraud (1:600)

**Метрики:**
- Precision, Recall, F1
- PR-AUC
- Business cost

---

## 📚 Часть 1: Теория

### 1.1 Проблема

**Imbalanced data:** Один класс редкий (minority) vs частый (majority)

**Проблемы:**
1. **Biased learning** - модель игнорирует minority
2. **Accuracy misleading** - 99% accuracy при предсказании только majority
3. **Gradient domination** - majority доминирует в градиентах

### 1.2 Решения

**3 подхода:**
1. **Data-level:** Resampling (SMOTE, undersampling)
2. **Algorithm-level:** Cost-sensitive learning, class weights
3. **Hybrid:** Комбинация обоих

---

### 1.3 SMOTE (Synthetic Minority Over-sampling)

**Идея:** Создать синтетические примеры minority класса

**Алгоритм:**
1. Для каждого minority примера $x_i$
2. Найти $k$ ближайших соседей (обычно k=5)
3. Случайно выбрать одного соседа $x_{nn}$
4. Создать синтетический пример:

$$x_{\text{new}} = x_i + \lambda (x_{nn} - x_i)$$

где $\lambda \sim \text{Uniform}(0, 1)$

**Интуиция:** Интерполяция между реальными примерами

**Преимущества:**
- ✅ Не дублирует примеры (vs naive oversampling)
- ✅ Расширяет decision region minority класса
- ✅ Уменьшает overfitting

**Недостатки:**
- ❌ Может создавать шум если классы перекрываются
- ❌ Не учитывает majority класс (может создать примеры в их области)

---

### 1.4 Cost-Sensitive Learning

**Идея:** Штрафовать модель больше за ошибки на minority

**Cost matrix:**

|  | Predict 0 | Predict 1 |
|--|-----------|----------|
| **Actual 0** | 0 | $C_{FP}$ |
| **Actual 1** | $C_{FN}$ | 0 |

**Total cost:**
$$\text{Cost} = C_{FP} \cdot FP + C_{FN} \cdot FN$$

**Class weights:**
Вместо минимизации loss, минимизируем weighted loss:

$$L_{\text{weighted}} = \sum_{i=1}^n w_{y_i} L(y_i, \hat{y}_i)$$

где 
$$w_0 = \frac{n}{n_0}, \quad w_1 = \frac{n}{n_1}$$

**Для 1:600 imbalance:**
- $w_0 = \frac{601}{600} \approx 1$
- $w_1 = \frac{601}{1} = 601$

Minority примеры получают вес 601!

---

### 1.5 Evaluation Metrics

**Accuracy НЕ подходит!**

Для imbalanced используем:

**1. Precision & Recall:**
$$\text{Precision} = \frac{TP}{TP + FP}, \quad \text{Recall} = \frac{TP}{TP + FN}$$

**2. F1-score:**
$$F_1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$

**3. PR-AUC:** Area под Precision-Recall кривой
- Лучше чем ROC-AUC для imbalanced!
- ROC-AUC может быть misleading

**4. Business Cost:**
$$\text{Cost} = C_{FP} \cdot FP + C_{FN} \cdot FN$$

---

## 📊 Часть 2: Практика

### 2.1 Импорты

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Imbalanced-learn
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek

# Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Utils
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve, roc_curve

plt.style.use('seaborn-v0_8-darkgrid')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
print('✅ Libraries loaded')

### 2.2 Dataset: Credit Card Fraud

**Kaggle Credit Card Fraud Detection**

- 284,807 транзакций
- 492 fraud (0.17%)
- 28 PCA features + Amount + Time
- Highly imbalanced!

Download: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

In [ ]:
# Загрузка
import os
data_path = '../../data/creditcard.csv'

if os.path.exists(data_path):
    df = pd.read_csv(data_path)
    print(f'✅ Loaded: {df.shape[0]:,} transactions')
    print(f'Features: {df.shape[1]}')
else:
    print('❌ File not found')
    print('Download from Kaggle and save to data/creditcard.csv')

### 2.3 EDA

In [ ]:
print('=== Info ===')
df.info()
print('\n=== Class distribution ===')
print(df['Class'].value_counts())
print('\nImbalance ratio:')
fraud_rate = df['Class'].mean()
print(f'Fraud: {fraud_rate:.4%}')
print(f'Ratio: 1:{int(1/fraud_rate)}')

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

df['Class'].value_counts().plot(kind='bar', ax=axes[0], color=['green', 'red'])
axes[0].set_title('Class Distribution (Extreme Imbalance!)')
axes[0].set_xlabel('Class (0=Normal, 1=Fraud)')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Normal', 'Fraud'], rotation=0)

for i, v in enumerate(df['Class'].value_counts()):
    axes[0].text(i, v + 5000, f'{v:,}\n({100*v/len(df):.2f}%)', ha='center')

df['Class'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.2f%%', colors=['green', 'red'])
axes[1].set_title('Class Proportion')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
# Features & target
X = df.drop('Class', axis=1)
y = df['Class']

print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

# Train/test split (stratified!)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y)

print(f'\nTrain: {X_train.shape[0]:,}')
print(f'Test: {X_test.shape[0]:,}')
print(f'\nFraud in train: {y_train.sum()} ({y_train.mean():.4%})')
print(f'Fraud in test: {y_test.sum()} ({y_test.mean():.4%})')

## 🎯 Часть 3: Baseline (No Resampling)

In [ ]:
def evaluate_fraud(model, X_tr, X_te, y_tr, y_te, name):
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)
    y_proba = model.predict_proba(X_te)[:, 1]
    
    prec = precision_score(y_te, y_pred)
    rec = recall_score(y_te, y_pred)
    f1 = f1_score(y_te, y_pred)
    pr_auc = average_precision_score(y_te, y_proba)
    
    # Business cost (FN = $500, FP = $10)
    tn, fp, fn, tp = confusion_matrix(y_te, y_pred).ravel()
    cost = fn * 500 + fp * 10
    
    print(f'\n{name}:')
    print(f'  Precision: {prec:.4f}')
    print(f'  Recall: {rec:.4f}')
    print(f'  F1: {f1:.4f}')
    print(f'  PR-AUC: {pr_auc:.4f}')
    print(f'  Confusion: TN={tn}, FP={fp}, FN={fn}, TP={tp}')
    print(f'  Cost: ${cost:,}')
    
    return {'model': model, 'prec': prec, 'rec': rec, 'f1': f1, 'pr_auc': pr_auc, 'cost': cost, 'y_pred': y_pred, 'y_proba': y_proba}

print('Eval function ready')

In [ ]:
results = {}

# Baseline XGBoost (no special handling)
xgb_baseline = XGBClassifier(n_estimators=100, max_depth=5, random_state=RANDOM_STATE, n_jobs=-1, eval_metric='logloss')
results['XGB_baseline'] = evaluate_fraud(xgb_baseline, X_train, X_test, y_train, y_test, 'XGBoost Baseline')

## 🔥 Часть 4: SMOTE

In [ ]:
print('Applying SMOTE...')
smote = SMOTE(random_state=RANDOM_STATE)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(f'Original: {y_train.value_counts().to_dict()}')
print(f'After SMOTE: {pd.Series(y_train_smote).value_counts().to_dict()}')
print(f'\nBalanced! Both classes have same count.')

In [ ]:
# XGBoost with SMOTE
xgb_smote = XGBClassifier(n_estimators=100, max_depth=5, random_state=RANDOM_STATE, n_jobs=-1, eval_metric='logloss')
results['XGB_SMOTE'] = evaluate_fraud(xgb_smote, X_train_smote, X_test, y_train_smote, y_test, 'XGBoost + SMOTE')

## ⚖️ Часть 5: Class Weights

In [ ]:
# Calculate scale_pos_weight
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f'scale_pos_weight: {scale_pos_weight:.0f}')

xgb_weighted = XGBClassifier(n_estimators=100, max_depth=5, scale_pos_weight=scale_pos_weight, random_state=RANDOM_STATE, n_jobs=-1, eval_metric='logloss')
results['XGB_weighted'] = evaluate_fraud(xgb_weighted, X_train, X_test, y_train, y_test, 'XGBoost + Class Weights')

## 🚀 Часть 6: Hybrid (SMOTE + Weights)

In [ ]:
# Moderate SMOTE (не до 50/50, а до 1:10)
from imblearn.over_sampling import SMOTE
smote_moderate = SMOTE(sampling_strategy=0.1, random_state=RANDOM_STATE)  # 10% instead of 50%
X_train_moderate, y_train_moderate = smote_moderate.fit_resample(X_train, y_train)

print(f'After moderate SMOTE:')
print(pd.Series(y_train_moderate).value_counts())
print(f'Ratio: 1:{(y_train_moderate==0).sum() / (y_train_moderate==1).sum():.0f}')

In [ ]:
# XGBoost with moderate SMOTE + weights
scale_moderate = (y_train_moderate == 0).sum() / (y_train_moderate == 1).sum()
xgb_hybrid = XGBClassifier(n_estimators=100, max_depth=5, scale_pos_weight=scale_moderate, random_state=RANDOM_STATE, n_jobs=-1, eval_metric='logloss')
results['XGB_hybrid'] = evaluate_fraud(xgb_hybrid, X_train_moderate, X_test, y_train_moderate, y_test, 'XGBoost + Hybrid')

## 📊 Часть 7: Comparison

In [ ]:
comp = pd.DataFrame({
    'Method': list(results.keys()),
    'Precision': [results[m]['prec'] for m in results],
    'Recall': [results[m]['rec'] for m in results],
    'F1': [results[m]['f1'] for m in results],
    'PR-AUC': [results[m]['pr_auc'] for m in results],
    'Cost ($)': [results[m]['cost'] for m in results]
}).sort_values('F1', ascending=False)

print('\n=== COMPARISON ===')
display(comp)
print(f'\n🏆 Best F1: {comp.iloc[0]["Method"]}')
print(f'💰 Lowest cost: {comp.loc[comp["Cost ($)"].idxmin(), "Method"]}')

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, metric in enumerate(['Recall', 'Precision', 'F1']):
    data = comp.sort_values(metric, ascending=False)
    axes[idx].barh(data['Method'], data[metric], color='skyblue')
    axes[idx].set_xlabel(metric, fontweight='bold')
    axes[idx].set_title(f'{metric} Comparison')
    axes[idx].invert_yaxis()
    for i, (m, v) in enumerate(zip(data['Method'], data[metric])):
        axes[idx].text(v + 0.01, i, f'{v:.3f}', va='center')

plt.tight_layout()
plt.show()

## 📝 Выводы

### Результаты

**Для fraud detection (1:600 imbalance):**

1. **Baseline** - низкий recall (пропускает fraud!)
2. **SMOTE** - улучшает recall, но снижает precision
3. **Class weights** - хороший баланс
4. **Hybrid** - лучший F1 и бизнес-метрики

### Рекомендации

**Для fraud detection:**
- ✅ Используйте class weights (обязательно!)
- ✅ Moderate SMOTE (1:10) + weights (лучший результат)
- ✅ Optimize threshold по business cost
- ✅ Мониторьте Recall (пропуски дороги!)

**Общие best practices:**
1. **Понимайте business cost** FP vs FN
2. **Используйте правильные метрики** (Precision-Recall, not Accuracy!)
3. **Stratified split** - обязательно
4. **Threshold optimization** - часто важнее чем модель
5. **Ensemble** - комбинируйте разные подходы

### Когда использовать что?

| Imbalance | Рекомендация |
|-----------|-------------|
| 1:4 (mild) | Class weights |
| 1:10-1:100 | SMOTE + class weights |
| 1:100+ (extreme) | Moderate SMOTE + weights + threshold opt |

---

**References:**
1. Chawla et al. (2002). SMOTE: Synthetic Minority Over-sampling Technique.
2. He & Garcia (2009). Learning from Imbalanced Data.
3. Lin et al. (2017). Focal Loss for Dense Object Detection.